In [4]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
dataDict = pickle.load(open('./data/pre_data/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [10]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))

In [11]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [12]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{공장, 건강, 남성들, 연기, 지역, 부작용, 세보 효과}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{건강, 철분, 치아, 자세, 입냄새, 식후, 점심 식사, 디스크, 낮잠, 커피}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{지인, 진술, 성관계, 아들, 앙심, 범행, 주변, 경찰, 남편, 철원}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{혐의, 신영자 이사장, 네이처리퍼블릭, 유통업체, 롯데, 검찰, 매장, 징역}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{혐의, 폭행, 쪽지, 범행, 과자, 남자친구, 경찰, 유치장}"


In [13]:
textInfo = df_daum.title.values + df_daum.mainText.values

In [14]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [15]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    return res

In [16]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

### doc2vec

In [17]:
d2v_docs = [nav_tokenizer(x) for x in textInfo]
d2v_docs = [remove_stopwords(x, (stopwords)) for x in d2v_docs]

In [76]:
tagged_d2v_docs = [TaggedDocument(doc, [idx]) for idx, doc in enumerate(d2v_docs)]

In [156]:
d2v_model = doc2vec.Doc2Vec(size=1000,  alpha=0.025, min_alpha=0.025, seed=0)
d2v_model.build_vocab(tagged_d2v_docs)
d2v_model.train_lbls = False # do not train labels of words
d2v_model.train_words = True # only train relations among words

In [157]:
for epoch in range(30):
    d2v_model.train(tagged_d2v_docs, total_examples=len(d2v_docs), epochs=epoch)
    d2v_model.alpha -= 0.002 # decrease the learning rate
    d2v_model.min_alpha = d2v_model.alpha # fix the learning rate, no decay

In [158]:
d2v_model.save('./data/pre_data/pre_data_classification_size1500_epoch30_doc2vec_model_by_mecab.d2v')

In [159]:
X_d2v = [d2v_model.infer_vector(x.words) for x in tagged_d2v_docs]

### word2vec

In [198]:
w2v_model = Word2Vec(d2v_docs, sg=1, size=2500,window=3)
w2v_model.init_sims(replace = True)

In [199]:
print(w2v_model.most_similar('신혜선'))

[('박시후', 0.9873740077018738), ('천호진', 0.9674841165542603), ('이진', 0.9366552233695984), ('유인영', 0.9365608096122742), ('서은수', 0.9197003245353699), ('장나라', 0.9095673561096191), ('셰어하우스', 0.8973545432090759), ('정소영', 0.896368145942688), ('재벌가', 0.8955332040786743), ('이태성', 0.8918585777282715)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [200]:
print(w2v_model.most_similar('손흥민'))

[('케인', 0.9378143548965454), ('토트넘', 0.9199273586273193), ('에릭센', 0.8644661903381348), ('공격수', 0.8415145874023438), ('맨시티', 0.8369077444076538), ('석현준', 0.8261291980743408), ('호날두', 0.8211312294006348), ('골', 0.8139941096305847), ('환상', 0.8113499879837036), ('이승우', 0.8061041831970215)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [201]:
print(w2v_model.most_similar('황금빛내인생'))

[('정소영', 0.9803756475448608), ('악녀', 0.9801917672157288), ('젠틀', 0.978105902671814), ('삼각관계', 0.977512776851654), ('김지석', 0.9772965908050537), ('이태성', 0.9770493507385254), ('소름', 0.9766101241111755), ('오연서', 0.9764827489852905), ('차승원', 0.976203441619873), ('브로맨스', 0.9760606288909912)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [202]:
print(w2v_model.most_similar('홍준표'))

[('당파', 0.9660859107971191), ('안철수', 0.9601312875747681), ('새누리당', 0.9549719095230103), ('박지원', 0.9535696506500244), ('김성태', 0.9494558572769165), ('천정배', 0.9489451050758362), ('서청원', 0.9427561163902283), ('자유한국당', 0.9419127106666565), ('바른정당', 0.9408918619155884), ('유승민', 0.9405010938644409)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [203]:
print(w2v_model.most_similar('문재인'))

[('방중', 0.8958843946456909), ('노무현', 0.8857604265213013), ('박정희', 0.8834023475646973), ('文', 0.8833121657371521), ('이명박', 0.8780871629714966), ('김영삼', 0.8658329844474792), ('국빈', 0.8608384132385254), ('주석', 0.8604637384414673), ('김대중', 0.8582534790039062), ('박근혜', 0.857179582118988)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [204]:
print(w2v_model.most_similar('노무현'))

[('文', 0.9809374809265137), ('김영삼', 0.9787030220031738), ('문재', 0.9693466424942017), ('수석비서관', 0.9660478830337524), ('김대중', 0.9660423994064331), ('장하성', 0.9633828401565552), ('좌파', 0.959345817565918), ('탄핵', 0.95714271068573), ('이명박', 0.954488217830658), ('국정', 0.9532121419906616)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [205]:
print(w2v_model.most_similar('박근혜'))

[('이명박', 0.9527423977851868), ('박정희', 0.9297676086425781), ('비서실장', 0.9220741987228394), ('탄핵', 0.9217817783355713), ('노무현', 0.9186800122261047), ('국정농단', 0.9158638715744019), ('국정', 0.9147265553474426), ('실세', 0.9079442024230957), ('비서관', 0.9051399827003479), ('정권', 0.9037990570068359)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [206]:
print(w2v_model.most_similar('국정원'))

[('서울중앙지검', 0.9500090479850769), ('문건', 0.9498414397239685), ('추명호', 0.9480745792388916), ('정무수석', 0.9465861320495605), ('감찰', 0.9444097280502319), ('민정수석', 0.9442946910858154), ('특활비', 0.9414039850234985), ('원세훈', 0.9403767585754395), ('국정원장', 0.9391836524009705), ('국정농단', 0.9360496401786804)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [207]:
print(w2v_model.most_similar('이낙연'))

[('안산시', 0.9813664555549622), ('안산', 0.9809675812721252), ('조례안', 0.9790827035903931), ('민주노총', 0.9779044985771179), ('신군부', 0.9772671461105347), ('기자실', 0.9750241041183472), ('칼둔', 0.9740843772888184), ('무안', 0.9739723205566406), ('발송', 0.9737409353256226), ('사무처장', 0.9737048745155334)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [208]:
print(w2v_model.most_similar('이국종'))

[('중증외상센터', 0.9124443531036377), ('아주대병원', 0.9003106355667114), ('병', 0.8894937038421631), ('정호균', 0.8815172910690308), ('이재갑', 0.8757880330085754), ('센터장', 0.875205934047699), ('기생충', 0.8679920434951782), ('창민', 0.865454912185669), ('수험', 0.8626434803009033), ('호소', 0.8620616793632507)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [209]:
print(w2v_model.most_similar('김성태'))

[('당파', 0.964714527130127), ('서청원', 0.9639352560043335), ('김무성', 0.961418628692627), ('새누리당', 0.9580477476119995), ('홍문종', 0.9553490877151489), ('김세연', 0.9549567699432373), ('원외', 0.9548509120941162), ('원내', 0.954719603061676), ('유성엽', 0.9530829191207886), ('한나라당', 0.9530700445175171)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [210]:
print(w2v_model.most_similar('더불어민주당'))

[('유성엽', 0.98927241563797), ('최경환', 0.9868556261062622), ('이정미', 0.9853346347808838), ('우원식', 0.9846246242523193), ('정동영', 0.9824989438056946), ('천정배', 0.9790178537368774), ('한나라당', 0.9779059886932373), ('사당', 0.9776094555854797), ('홍문표', 0.9764937162399292), ('선출', 0.9763784408569336)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [211]:
print(w2v_model.most_similar('새누리당'))

[('친박', 0.9778624773025513), ('당파', 0.9724947214126587), ('서청원', 0.9715394973754883), ('친박계', 0.9714686274528503), ('홍문종', 0.970689058303833), ('선출', 0.9704322814941406), ('천정배', 0.9695726633071899), ('당대표', 0.9690241813659668), ('사당', 0.968609631061554), ('원내', 0.9680299162864685)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [212]:
print(w2v_model.most_similar('자유한국당'))

[('국민의당', 0.965130090713501), ('안철수', 0.955971360206604), ('원내대표', 0.9529104232788086), ('한국당', 0.9505704641342163), ('홍준표', 0.9419127106666565), ('국회의원', 0.9353947043418884), ('바른정당', 0.9330065250396729), ('당파', 0.9324742555618286), ('민주당', 0.9298559427261353), ('새누리당', 0.9199388027191162)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [213]:
print(w2v_model.most_similar('국민의당'))

[('자유한국당', 0.965130090713501), ('안철수', 0.9575421810150146), ('한국당', 0.9547532796859741), ('당', 0.9464746713638306), ('민주당', 0.942284345626831), ('정당', 0.941257119178772), ('바른정당', 0.9324765205383301), ('국회의원', 0.9324659109115601), ('원내대표', 0.9316573143005371), ('당원', 0.9240709543228149)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [214]:
print(w2v_model.most_similar('안철수'))

[('바른정당', 0.9798340201377869), ('유승민', 0.9717925786972046), ('당파', 0.9648303389549255), ('홍준표', 0.9601312875747681), ('새누리당', 0.9599226713180542), ('국민의당', 0.9575421810150146), ('자유한국당', 0.955971360206604), ('정의당', 0.9516004323959351), ('친박', 0.9496731758117676), ('야권', 0.946500301361084)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [215]:
print(w2v_model.most_similar('유승민'))

[('바른정당', 0.9890029430389404), ('안철수', 0.9717925786972046), ('당파', 0.9648632407188416), ('야권', 0.9641416072845459), ('새누리당', 0.9600166082382202), ('당내', 0.9582798480987549), ('친박계', 0.9582614898681641), ('김무성', 0.9566706418991089), ('정의당', 0.9563947916030884), ('친박', 0.955491304397583)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [99]:
w2v_model.save('./data/pre_data/word2vec_category_size1500_by_mecab.w2v')